In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp ./drive/MyDrive/image_set.zip ./
!unzip ./image_set.zip

Archive:  ./image_set.zip
   creating: image_set/
  inflating: image_set/1.jpg         
  inflating: image_set/2.jpg         
  inflating: image_set/3.jpg         
  inflating: image_set/4.jpg         
  inflating: image_set/5.jpg         
  inflating: image_set/6.jpg         


In [ ]:
import cv2
import numpy as np

# Function to convert an image to grayscale and normalize pixel values
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    normalized_image = gray_image.astype(np.float32) / 255.0
    resized_image = cv2.resize(normalized_image, (1008, 1344))

    return resized_image

# List to store preprocessed images
preprocessed_images = []

# Open and preprocess each image
image_paths = ["./1.jpg", "./2.jpg", "./3.jpg", "./4.jpg", "./5.jpg", "./6.jpg", "./7.jpg"]
# image_paths = ["./1.jpg", "./2.jpg"]
for path in image_paths:
    image = cv2.imread(path)
    preprocessed_image = preprocess_image(image)
    preprocessed_images.append(preprocessed_image)

max_width = max(image.shape[1] for image in preprocessed_images)
max_height = max(image.shape[0] for image in preprocessed_images)

# Create the 3D matrix to store the stacked images
stacked_image = np.zeros((len(preprocessed_images), max_height, max_width), dtype=np.float32)

# Stack images vertically while preserving resolution
for i, image in enumerate(preprocessed_images):
    stacked_image[i, :image.shape[0], :image.shape[1]] = image

# Save the stacked image as npy file
np.save("stacked_image_small.npy", stacked_image)

In [ ]:
(stacked_image.shape)

(7, 1344, 1008)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def fit_curve_and_find_max_rate(points):
    # Generate x and y coordinates from the given points
    x = np.arange(len(points))
    y = np.array(points)

    # Fit a polynomial curve of degree 5 to the points
    coefficients = np.polyfit(x, y, 5)
    polynomial = np.poly1d(coefficients)

    # Calculate the derivative of the polynomial equation
    derivative = np.polyder(polynomial)

    # Find the critical points where the derivative is zero or undefined
    critical_points = np.roots(derivative)

    # Evaluate the derivative at the critical points to find the maximum rate of change
    max_rate_of_change = np.max(np.abs(np.polyval(derivative, critical_points)))

    # Find the point with maximum rate of change
    max_rate_x = critical_points[np.argmax(np.abs(np.polyval(derivative, critical_points)))]
    max_rate_y = polynomial(max_rate_x)
    return polynomial, (max_rate_x, max_rate_y), max_rate_of_change


In [ ]:
import numpy as np
!cp /content/drive/MyDrive/stacked_image_small.npy /content
stacked_image = np.load('./stacked_image_small.npy')

In [ ]:
print(stacked_image.shape)

(6, 1344, 1008)


In [ ]:
matrix_height = len(stacked_image[0,:,0])
matrix_width = len(stacked_image[0,0,:])
print(matrix_height)
print(matrix_width)

1344
1008


In [ ]:
import numpy as np

matrix_height = len(stacked_image[0,:,0])
matrix_width = len(stacked_image[0,0,:])
final_image_translator = np.zeros((matrix_height, matrix_width))
max_rate_change_matrix = np.zeros((matrix_height, matrix_width))



for row in range(matrix_height):
  for col in range(matrix_width):

    first_pixel_values = stacked_image[:, row, col]
    # Fit the curve and find the local maxima
    polynomial, max_rate_point, max_rate_of_change = fit_curve_and_find_max_rate(first_pixel_values)
    max_x, max_y = max_rate_point
    final_image_translator[row, col] = max_x
    max_rate_change_matrix[row, col] = max_rate_of_change
  print(row)

# Print the polynomial equation coefficients
#print("Polynomial Equation:")
#print(polynomial)

In [ ]:
print(max_rate_change_matrix[10,10])

1.2490310190672261e-16


In [ ]:
# to find the average rate of change across the whole set of seven images
import matplotlib.pyplot as plt

rate_sum = 0
rate_div = 0

for row in range(matrix_height):
  for col in range(matrix_width):
    rate_sum = max_rate_change_matrix[row, col]
    rate_div += 1

avg_rate_change = rate_sum / rate_div


In [ ]:
# cleaning the image translation matrix
matrix_height = len(stacked_image[0,:,0])
matrix_width = len(stacked_image[0,0,:])

for row in range(matrix_height):
    for col in range(matrix_width):
      if final_image_translator[row, col] <= 0:
        final_image_translator[row, col] = 0

      elif final_image_translator[row, col] >= 6:
        final_image_translator[row, col] = 6


In [ ]:
import matplotlib.pyplot as plt

#variables
matrix_height = len(stacked_image[0,:,0])
matrix_width = len(stacked_image[0,0,:])
image_depth = 0
final_image = np.zeros((matrix_height, matrix_width))
print(final_image.shape)
for row in range(matrix_height):
    for col in range(matrix_width):
      image_depth = int(final_image_translator[row, col])
      if max_rate_change_matrix[row, col] > 100000000 * avg_rate_change: #important parameter
        image_depth = 0
      final_image[row, col] = stacked_image[image_depth, row, col]

final_final_image = final_image * 255
filename = 'fused.jpg'
cv2.imwrite(filename, final_final_image)

(1344, 1008)


True

In [ ]:
filename = 'fused.jpg'
cv2.imwrite(filename, final_final_image)

True

In [ ]:
###############################################################################
###############################################################################
###############################################################################
# parallalizing the code using CUDA